In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from keras import regularizers
import tensorflow as tf


In [ ]:
#input
Data_set= pd.read_csv("/home/effrancodelos/Deep_learnig/Data_Nova/COAD/COAD_COX/data_Merge_Colon_t.csv")
Data_set.head(n=20)

In [ ]:
#Create a data-set with the string columns

Pacients = Data_set['Pacients']




In [ ]:
#Remove string column
Data_set.drop(['Pacients'], axis=1, inplace=True)


In [ ]:
Data_set_scaled =minmax_scale(Data_set, axis = 0)
#Data_set_scaled = StandardScaler().fit_transform(Data_set)

In [ ]:
ncol = Data_set_scaled.shape[1]




In [ ]:
#split the data set 0.8 for training and 20% for test
X_train, X_test = train_test_split(Data_set_scaled, train_size = 0.8, random_state = seed(2017))


In [ ]:
len(X_test)

In [ ]:
encoding_dim = 100

In [ ]:
input_dim = Input(shape = (ncol, ))
# Encoder Layers

encoded2 = Dense(500, activation = 'tanh',activity_regularizer=regularizers.l1_l2(l1=0.000001,l2=0.00001))(input_dim)
#encoded3 = Dense(500, activation = 'relu',activity_regularizer=regularizers.l1(10e-5))(input_dim)
encoded3 = Dense(encoding_dim, activation = 'tanh')(encoded2)

# Decoder Layers

#decoded1 = Dense(500, activation = 'relu')(encoded3)
decoded1 = Dense(500, activation = 'tanh',activity_regularizer=regularizers.l1_l2(l1=0.000001, l2=0.00001))(encoded3)
decoded2 = Dense(ncol, activation = 'sigmoid')(decoded1)

In [ ]:
# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded2)

In [ ]:
# Compile the Model
autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy')
#autoencoder.compile(optimizer = 'adam', loss = 'mean_squared_error')# BEST

In [ ]:
autoencoder.summary()

In [ ]:
#fit the model 
autoencoder.fit(X_train, X_train, epochs = 50, batch_size = 100, shuffle = False, validation_data = (X_test, X_test))

In [ ]:
encoder = Model(inputs = input_dim, outputs = encoded3)
encoded_input = Input(shape = (encoding_dim, ))

In [ ]:
#Get the output
encoded_train = pd.DataFrame(encoder.predict(Data_set_scaled))
encoded_train = encoded_train.add_prefix('feature_')

In [ ]:
encoded_train['Pacients'] = Pacients
#encoded_train['Radiations'] = Radiation

In [ ]:
print(encoded_train.shape)
encoded_train.head()

In [ ]:
encoded_train.to_csv("/home/effrancodelos/Deep_learnig/Data_Nova/COAD/COAD_COX/data_Merge_Colon_COX_Spare_1.csv", index=False)